# Downcasting

## Options & Imports

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [ ]:
# Import Libraries & Parameters
import os
import pandas as pd
import numpy as np
from config import gcp_token, bq_db, project_id

In [ ]:
# Set GCP credentials
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = gcp_token

### Import from BQ

We'll start by importing a df from Google Big Query. The Date field is transformed as BigQuery converts everything to UTC format which interferes with Pandas datetime operations.

TODO: Change this to a .csv

In [ ]:
# Our SQL query
query = f'SELECT * FROM {bq_db}.merged'

# Import the data from BQ
df = pd.read_gbq(query=query, project_id=project_id)

# Transform the date column
df['date'] = df['date'].dt.tz_localize(None)

# Show the df
df.head()

### Downcasting

Pandas will generally default to 64-bit storage for data. This means that it's more versatile at the expense of taking up memory, which in turn generally makes things longer to process.

Converting this to a more efficient storage mechanism for data is called **Downcasting**. Downcasting is very quick to do and can give a welcome efficiency boost when dealing with large dataframes.

We can check how much memory is being used with the `memory_usage()` method.

In [ ]:
df.memory_usage()

In [ ]:
df.memory_usage().sum()

And we can check the data types with the `dtypes()` method:

In [ ]:
df.dtypes

### Downcasting a Dataframe

We can downcast our dataframe programatically using `dtype` and `astype` as follows:

In [ ]:
df_dc = df.copy()

# Get float & int cols
float_cols = [col for col in df_dc if df_dc[col].dtype in ["float32", "float64"]]
int_cols = [col for col in df_dc if df_dc[col].dtype in ["int64", "int32"]]

# Downcast float & int cols to 16 bit numpy formats
df_dc[float_cols] = df_dc[float_cols].astype(np.float16)
df_dc[int_cols] = df_dc[int_cols].astype(np.int16)

In [ ]:
int_cols

In [ ]:
df.memory_usage().sum()

In [ ]:
df_dc.memory_usage().sum()

### Further Reading

* [Advanced Pandas optimize Speed & Memory](https://medium.com/bigdatarepublic/advanced-pandas-optimize-speed-and-memory-a654b53be6c2)
* [Downcasting in Pandas](https://medium.com/@vincentteyssier/optimizing-the-size-of-a-pandas-dataframe-for-low-memory-environment-5f07db3d72e)